In [ ]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

from pathlib import Path
from kedro.framework.context import load_context

current_dir = Path.cwd()  # this points to 'notebooks/' folder
proj_path = current_dir.parent  # point back to the root of the project
context = load_context(proj_path)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [ ]:
df = context.catalog.load('fp_projections_remote')
df.head()

In [ ]:
df.query('position == "rb"')

In [ ]:
import pandas as pd

def s(uri, path, **a):
    return pd.read_html(uri, path, **a)

df = s('https://www.pro-football-reference.com/years/2019/rushing.htm','Rushing Table',skiprows=1)


In [ ]:
df = pd.read_csv('https://github.com/DynastyProcess/data/blob/master/files/db_fpecr.csv')
df.head()


#df = pd.read_parquet()

In [ ]:
df.groupby('fantasypros_id').head()

In [ ]:
df_rb = pd.read_html('https://www.cbssports.com/fantasy/football/stats/RB/2020/season/projections/nonppr/',header=1)[0]
df_wr = pd.read_html('https://www.cbssports.com/fantasy/football/stats/WR/2020/season/projections/nonppr/',header=1)[0]
df_te = pd.read_html('https://www.cbssports.com/fantasy/football/stats/TE/2020/season/projections/nonppr/',header=1)[0]
df_qb = pd.read_html('https://www.cbssports.com/fantasy/football/stats/QB/2020/season/projections/nonppr/',header=1)[0]

In [ ]:
df_flex = pd.concat([df_rb, df_wr, df_te],ignore_index=True)

In [ ]:
df_flex.columns

In [ ]:
df_names = pd.DataFrame()

df_names = df_flex.Player.str.extract(r'(?P<stuff>.*?)(?P<pos>[A-Z]{1,2})\s*(?P<team>[A-Z]{2,3}$)')
df_names[['stuff_2','name']] = df_names.stuff.str.extract(r'(?P<stuff2>[A-Z]{2}\s*[A-Z]{2,3})\s*(?P<name>.*)')
df_names.drop(columns=['stuff','stuff_2'], inplace=True)
df_names

In [ ]:
df_3 = df_names[['pos','team']]
df_3.team.str.rstrip()

In [ ]:
import pandas as pd

df1 = pd.DataFrame({'Player':['A','B','C'], 
    'Stat_1':range(3),'Stat_2':range(3),'Stat_3':range(3),
    'pos':['rb','qb','te']})
df2 = pd.DataFrame({'Player':['A','B','C'], 'Stat_1':range(3,6),'Stat_2':range(3,6),'Stat_3':range(3,6),
'pos':['rb','qb','te'] })
df3 = pd.DataFrame({'Player':['A','B','C'], 'Stat_1':range(3,6),'Stat_2':range(3,6),
'pos':['rb','qb','te'] })
df1['source'] ='fp'
df2['source'] ='cbs'
df3['source'] ='other'

# df1 = df1.set_index('Player')
# df2 = df2.set_index('Player')
# df3 = df3.set_index('Player')
df1
df2
df3

In [ ]:
def avg_ds(*dataframes):
    count = len(dataframes)
    df_temp = sum([d.select_dtypes('number') for d in dataframes])
    return df_temp

def mean_ds(*dataframes):
    combined_dataframes = pd.DataFrame()
    for d in dataframes:
        combined_dataframes = pd.concat([combined_dataframes, d])
    combined_dataframes.groupby('Player').mean()
    return combined_dataframes

df4 = mean_ds(df1,df2, df3)
df4



In [ ]:
combine_ds
df4 = pd.concat([df1, df2, df3])

df5 = df4.groupby('Player').mean()
#df5['pos'] = df1['pos']
df5

In [ ]:
df_cbs = context.catalog.load('cbs_projections_local')
df_fp = context.catalog.load('fp_projections_local')
df_cbs.head()

In [ ]:
def combine_ds(*dataframes):
    df_all = pd.concat(dataframes)
    df_all = df_all.groupby(['player','team','position']).mean().fillna(0)
    df_all = df_all[df_all.sum(axis=1) > 0].reset_index()
    return df_all
    # df_player = dataframes[0][['player','team','position']]
    # df_player = df_player.set_index(['player','team','position'])
    
    #return df_all.set_index(['player','team','position'])

combine_ds(df_cbs, df_fp)
combine_ds(df_cbs)



In [ ]:
df_cbs.join(df_fp, on=['player','team','position'])